In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [2]:
import kagglehub

# Download latest version
def download_deepfake_images():
    return kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

path = download_deepfake_images()

print("Path to dataset files:", path)

C:\Users\najee\projects\deepfake-detection-system\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\najee\.cache\kagglehub\datasets\manjilkarki\deepfake-and-real-images\versions\1


In [3]:
import os

train_path = os.path.join(path, "Dataset/Train")
test_path = os.path.join(path, "Dataset/Test")
validation_path = os.path.join(path, "Dataset/Validation")

In [4]:
from torchvision import datasets, transforms

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(150),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

test_transform = transforms.Compose([
    transforms.Resize(150),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

training_dataset = datasets.ImageFolder(train_path, transform=train_transform)
testing_dataset = datasets.ImageFolder(test_path, transform=test_transform)
validation_dataset = datasets.ImageFolder(validation_path, transform=test_transform)

In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    training_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
)
test_loader = DataLoader(
    testing_dataset,
    batch_size=64,
    num_workers=8,
    pin_memory=True,
)
validation_loader = DataLoader(
    validation_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
)

In [9]:
import timm

model = timm.create_model(
    "efficientnet_b0",
    pretrained=True,
    num_classes=2
).to(device)

In [10]:
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [11]:
from torchsummary import summary

summary(model, input_size=(3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 75, 75]             864
          Identity-2           [-1, 32, 75, 75]               0
              SiLU-3           [-1, 32, 75, 75]               0
    BatchNormAct2d-4           [-1, 32, 75, 75]              64
            Conv2d-5           [-1, 32, 75, 75]             288
          Identity-6           [-1, 32, 75, 75]               0
              SiLU-7           [-1, 32, 75, 75]               0
    BatchNormAct2d-8           [-1, 32, 75, 75]              64
          Identity-9           [-1, 32, 75, 75]               0
           Conv2d-10              [-1, 8, 1, 1]             264
             SiLU-11              [-1, 8, 1, 1]               0
           Conv2d-12             [-1, 32, 1, 1]             288
          Sigmoid-13             [-1, 32, 1, 1]               0
    SqueezeExcite-14           [-1, 32,

In [12]:
import torch.nn as nn
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [15]:
def train(model, optimizer, criterion, device="cpu", epochs=5):
    model.train()
    dataset_size = len(train_loader.dataset)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs} -----------------------------------")
        for batch_idx, (X, y) in enumerate(train_loader):
            optimizer.zero_grad()

            X, y = X.to(device), y.to(device)

            logits = model(X)
            loss = criterion(logits, y)

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                loss, current = loss.item(), batch_idx * len(X)
                print(f"Loss {loss:.4f} [{current}/{dataset_size}]")

    print("Finished Training")

In [16]:
train(model, optimizer, criterion, device=device, epochs=5)

Epoch 1/5 -----------------------------------
Loss 3.7677 [0/140002]
Loss 2.3017 [6400/140002]
Loss 2.1654 [12800/140002]
Loss 1.5667 [19200/140002]
Loss 1.6000 [25600/140002]
Loss 1.6654 [32000/140002]
Loss 1.0075 [38400/140002]
Loss 0.9344 [44800/140002]
Loss 1.2759 [51200/140002]
Loss 1.2478 [57600/140002]
Loss 0.6744 [64000/140002]
Loss 1.0723 [70400/140002]
Loss 0.9550 [76800/140002]
Loss 1.3751 [83200/140002]
Loss 0.6855 [89600/140002]
Loss 0.9798 [96000/140002]
Loss 1.2565 [102400/140002]
Loss 0.6755 [108800/140002]
Loss 0.7791 [115200/140002]
Loss 0.8663 [121600/140002]
Loss 0.6966 [128000/140002]
Loss 0.5927 [134400/140002]
Epoch 2/5 -----------------------------------
Loss 1.0289 [0/140002]
Loss 0.5441 [6400/140002]
Loss 0.9269 [12800/140002]
Loss 0.5978 [19200/140002]
Loss 0.7191 [25600/140002]
Loss 0.5932 [32000/140002]
Loss 0.8096 [38400/140002]
Loss 0.5193 [44800/140002]
Loss 0.6164 [51200/140002]
Loss 0.5705 [57600/140002]
Loss 0.3975 [64000/140002]
Loss 0.7518 [70400/14

In [18]:
import torchmetrics

accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=2).to(device)

accuracy.reset()

model.eval()
with torch.no_grad():
    for X, y in validation_loader:
        X, y = X.to(device), y.to(device)
        logits = model(X)

        accuracy.update(logits, y)

result_accuracy = accuracy.compute()

print(result_accuracy.item())

0.7292786836624146


In [19]:
torch.save(model.state_dict(), "model.pth")